# yolo loss

In [2]:
import keras
from keras.preprocessing import image
import numpy as np
import os
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from scipy.misc import imread, imresize
%matplotlib inline 
import matplotlib.patches as patches
import xml.etree.ElementTree as ET

Using TensorFlow backend.


In [3]:
y = np.array([[0,0,0,0,0, 1, 0.2, 0.3, 0.05, 0.6]])

In [2]:
y_ = np.array([[0.123,0.01,0.2,0.05,0.05, 0.8, 0.3, 0.2, 0.5, 0.3]])

In [11]:
keras.losses.mean_squared_error(y, y_)

<tf.Tensor 'Mean:0' shape=(1,) dtype=float64>

In [45]:
a = np.array([[1, 2], [0,0]])
matcher = a[...,0] == 1
print matcher
print "select by matcher", a[matcher]

[ True False]
select by matcher [[1 2]]


In [47]:
# calc the loss 
y = np.array([[0,0,0,0,0, 1,0.1,0.2,0.3,0.4]])
y_ = np.array([[0.5,0,0,0,0, 1,0.4,0.3,0.2,0.1]])
output_shape = y.shape
print "output shape", output_shape

# reshape to split into boxes
y_ = y_.reshape([-1, 5])
y = y.reshape([-1, 5])

print "y reshaped into", y
print "y_ reshaped into", y_

objects_matcher = y[..., 0] == 1.0
print "objects_matcher", objects_matcher
objects_boxes = y[objects_matcher][..., 1:5]
objects_boxes_ = y_[objects_matcher][..., 1:5]
print "object boxes", objects_boxes
print "object boxes_", objects_boxes_
boxes_loss = np.sum(np.square(objects_boxes - objects_boxes_))
print "object loss", boxes_loss

boxes = np.concatenate((y[..., 0:5]), axis=-1)
boxes_ = np.concatenate((y_[..., 0:5]), axis=-1)
# print "object boxes data", boxes[objects == 1.]
# print "object boxes_ data", boxes_[objects == 1.]

print "cacl the confidence values"
confidences = y[..., 0]
confidences_ = y_[..., 0]
print "confiences", confidences
print "confiences_", confidences_

print ""
objects_confidences = confidences
objects_confidences_ = confidences_[confidences == 1.0]
print "object confidence", objects_confidences
print "object confidence_", objects_confidences_
object_confidence_loss = np.sum(np.square(objects_confidences - objects_confidences_))
print "loss for object confience", object_confidence_loss

print ""
non_objects_confidences_ = confidences_[confidences == .0]
print "non object confidence_", non_objects_confidences_
non_object_confidence_loss = np.sum(np.square(non_objects_confidences_))
print "loss for non object confience", non_object_confidence_loss



output shape (1, 10)
y reshaped into [[ 0.   0.   0.   0.   0. ]
 [ 1.   0.1  0.2  0.3  0.4]]
y_ reshaped into [[ 0.5  0.   0.   0.   0. ]
 [ 1.   0.4  0.3  0.2  0.1]]
objects_matcher [False  True]
object boxes [[ 0.1  0.2  0.3  0.4]]
object boxes_ [[ 0.4  0.3  0.2  0.1]]
object loss 0.2
cacl the confidence values
confiences [ 0.  1.]
confiences_ [ 0.5  1. ]

object confidence [ 0.  1.]
object confidence_ [ 1.]
loss for object confience 1.0

non object confidence_ [ 0.5]
loss for non object confience 0.25


In [97]:
import numpy as K

def yolo_loss(y_, y, confidence_loss_weight = 1.0, location_loss_weight = 1.0, noobj_loss_weight = 1.0, grid = (5,5), print_fn = None):
  # reshape to split into boxes
  y_ = K.reshape(y_, [-1, grid[0] * grid[1], 5])
  y = K.reshape(y, [-1,  grid[0] * grid[1], 5])

  objects_matcher = y[..., 0:1]
  objects_boxes = y[..., 1:5] * objects_matcher
  objects_boxes_ = y_[..., 1:5] * objects_matcher
  boxes_loss = K.mean(K.square(objects_boxes - objects_boxes_), axis=-1) * location_loss_weight

  objects_confidences = y[..., 0:1]
  objects_confidences_ = y_[..., 0:1] * objects_confidences
  object_confidence_loss = K.mean(K.square(objects_confidences - objects_confidences_), axis=-1) * confidence_loss_weight

  non_objects_confidences_ = y_[..., 0:1] * (1.0 - y[..., 0:1])
  non_object_confidence_loss = K.mean(K.square(non_objects_confidences_), axis=-1) * noobj_loss_weight
  
  total_loss = boxes_loss  + object_confidence_loss  + non_object_confidence_loss 

  if print_fn:
        print_fn("total loss {}, box loss {} confidence loss {} non object confidence loss {}".format(total_loss, boxes_loss, object_confidence_loss, non_object_confidence_loss))
  return K.sum(total_loss)

In [98]:
y = np.array([[0,0,0,0,0,1,0.1,0.2,0.3,0.4]])
y_ = np.array([[0.5,0,0,0,0, 1,0.4,0.3,0.2,0.1]])
def p(str):
    print str
print yolo_loss(y_, y, grid = (1,1), print_fn = p)

total loss [[ 0.25]
 [ 0.05]], box loss [[ 0.  ]
 [ 0.05]] confidence loss [[ 0.]
 [ 0.]] non object confidence loss [[ 0.25]
 [ 0.  ]]
0.3
